# TUTORIAL: LIMPIEZA DE DATOS CON PYTHON Y PANDAS

# 1. El problema del negocio

Una entidad bancaria contrata a una empresa de marketing encargada de contactar telefónicamente a posibles clientes para determinar si están interesados o no en adquirir un certificado de depósito a término con el banco.

¿Qué perfil tienen los clientes con mayor potencial de conversión?

#2. El set de datos

La información recolectada por la empresa de mercadeo se encuentra en un archivo CSV (`dataset_banco.csv`) con 45215 filas y 17 columnas.

Cada registro contiene 16 características (las primeras 16 columnas) y una categoría ("yes" o "no" dependiendo de si la persona está o no interesada en adquirir el producto). Las columnas son:

1. "age":  edad (numérica)
2. "job": tipo de trabajo (categórica: "admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student", "blue-collar","self-employed", "retired", "technician", "services")
3. "marital": estado civil (categórica: "married", "divorced", "single")
4. "education": nivel educativo (categórica: "unknown", "secondary", "primary", "tertiary")
5. "default": si dejó de pagar sus obligaciones (categórica: "yes", "no")
6. "balance": saldo promedio anual en euros (numérica)
7. "housing": ¿tiene o no crédito hipotecario? (categórica: "yes", "no")
8. "loan": ¿tiene créditos de consumo? (categórica: "yes", "no")
9. "contact": medio a través del cual fue contactado (categórica: "unknown", "telephone", "cellular")
10. "day": último día del mes en el que fue contactada (numérica)
11. "month": último mes en el que fue contactada (categórica: "jan", "feb", "mar", ..., "nov", "dec")
12. "duration": duración (en segundos) del último contacto (numérica)
13. "campaign": número total de veces que fue contactada durante la campaña (numérica)
14. "pdays": número de días transcurridos después de haber sido contactado antes de la campaña actual (numérica. -1 indica que no fue contactado previamente)
15. "previous": número de veces que ha sido contactada antes de esta campaña (numérica)
16. "poutcome": resultado de la campaña de marketing anterior (categórica: "unknown", "other", "failure", "success")
17. "y": categoría ¿el cliente se suscribió a un depósito a término? (categórica: "yes", "no")

#3. Una primera mirada al dataset

In [4]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# Lectura
from google.colab import drive

ruta = "/content/dataset_banco.csv"
data = pd.read_csv(ruta)

In [6]:
print(data.shape)
data.head()

(25340, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1.0,0.0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1.0,0.0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1.0,0.0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1.0,0.0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1.0,0.0,unknown,no


In [7]:
# Veamos las variables categóricas y las numéricas
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25340 entries, 0 to 25339
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        25340 non-null  int64  
 1   job        25340 non-null  object 
 2   marital    25340 non-null  object 
 3   education  25340 non-null  object 
 4   default    25340 non-null  object 
 5   balance    25340 non-null  int64  
 6   housing    25340 non-null  object 
 7   loan       25340 non-null  object 
 8   contact    25340 non-null  object 
 9   day        25340 non-null  int64  
 10  month      25340 non-null  object 
 11  duration   25340 non-null  int64  
 12  campaign   25340 non-null  int64  
 13  pdays      25339 non-null  float64
 14  previous   25339 non-null  float64
 15  poutcome   25339 non-null  object 
 16  y          25339 non-null  object 
dtypes: float64(2), int64(5), object(10)
memory usage: 3.3+ MB


#4. Limpieza

Realizaremos el proceso de limpieza teniendo en cuenta las situaciones más comunes:

1. Datos faltantes en algunas celdas
2. Columnas irrelevantes (que no responden al problema que queremos resolver)
3. Registros (filas) repetidos
4. Valores extremos (*outliers*) en el caso de las variables numéricas. Se deben analizar en detalle pues no necesariamente la solución es eliminarlos
5. Errores tipográficos en el caso de las variables categóricas

Al final de este proceso de limpieza deberíamos tener un set de datos **íntegro**, listo para la fase de Análisis Exploratorio.

## 4.1 Datos faltantes

Comenzamos a ver que los datos no están completos, pues no todas las columnas tienen la misma cantidad de registros.

El número total de registros debería ser 45.215. Sin embargo columnas como "job", "marital", "education", "balance", "duration" y "pdays".

Por ser tan pocos los datos  faltantes optaremos por eliminar las filas correspondientes:

##4.2 Columnas irrelevantes

Una columna irrelevante puede ser:

- **Una columna que no contiene información relevante para el problema que queremos resolver**. Por ejemplo en este caso podría ser una columna que no guarde relación con el posible perfil del cliente (deporte favorito, hobbies, comida favorita, etc.)
- **Una columna categórica pero con un sólo nivel**. Por ejemplo si en la columna "job" solo tuviésemos el nivel "unknown".
- **Una columna numérica pero con un sólo valor**. Por ejemplo si en la columna "edad" todos los vlaores fuesen iguales a 50.
- **Columnas con información redundante**. Por ejemplo si además de las columnas "month" y "day" tuviésemos la columna "month-day", resultado de combinar las dos anteriores.

Si tenemos la duda de si alguna columna puede ser relevante o no lo mejor es dejarla (y más adelante en posteriores etapas podremos darnos cuenta de si se debe preservar o no).

En este caso todas las columnas pueden resultar relevantes, pero debemos verificar que no haya columnas categóricas con un sólo nivel, o columnas numéricas con un sólo valor:

Todas las columnas categóricas tienen más de 1 subnivel. No eliminaremos ninguna.

Verifiquemos lo que ocurre con las columnas numéricas:

Todas las columnas numéricas tienen desviaciones estándar ("std") diferentes de cero, lo que indica que no tienen un único valor.

Preservaremos todas las columnas numéricas.

##4.3 Filas repetidas

##4.4 *Outliers* en las variables numéricas

No siempre se deben eliminar los *outliers* porque dependiendo de la variable numérica analizada estos pueden contener información importante.


##4.5 Errores tipográficos en variables categóricas

En una variable categórica pueden aparecer sub-niveles como "unknown" y "UNK" que para nosotros son equivalentes pero que para nuestro programa parecerían diferentes.

Se deben unificar estos sub-niveles

## Conclusiones

In [8]:
data.to_csv(ruta, index=False)

In [35]:
import pandas as pd

#Se ha realizado, anteriormente, una búsqueda con "unique", que visualiza los diferentes campos de una misma columna.
df = pd.read_csv("dataset_banco.csv")
df['age'] = pd.to_numeric(df['age'], errors='coerce') #Los no numéricos se convierten a NaN

df = df.dropna(subset=['age'])

df = df[df['age'] <= 101]
df["age"] = df["age"].astype(int)

valores_unicos_age = sorted(df["age"].unique())

print(valores_unicos_age)

In [31]:
#La siguiente configuración es un segundo ejemplo de cómo podría realizarse la limpieza pero requiere de más repetición de código.
#df = pd.read_csv("dataset_banco.csv")

#df = df.replace("MANAGEMENT", "management")
#df = df.replace("Management", "management")
#df = df.replace("Retired", "retired")
#df = df.replace("admin.", "administration")
#df = df.replace("administrative", "administration")
#df = df.replace("Services", "services")
#df = df.replace("Self-employed", "self-employed")

#df.to_csv("dataset_banco_modificado.csv", index=False)

#valores_unicos_job = df["job"].unique()
#print(valores_unicos_job)

job = {
    'MANAGEMENT': 'management',
    'Management': 'management',
    'Retired': 'retired',
    'administrative': 'admin.',
    'Services': 'services',
    'Self-employed': 'self-employed',
}

data['job'] = data['job'].replace(job)

In [11]:
marital = {
    'Single': 'single',
    'div.': 'divorced',
    'DIVORCED': 'divorced',
}

data['marital'] = data['marital'].replace(marital)

['married' 'single' 'divorced']


In [34]:
education = {
    'UNK': 'unknown',
    'Primary': 'primary',
    'SECONDARY': 'secondary',
    'sec.': 'secondary',
    'Secondary': 'secondary',
    'Tertiary': 'tertiary',
}

data['education'] = data['education'].replace(education)

Las columnas *default*, *balance*, *housing*, *day*, *month*, *duration*, *campaign*, *pdays* y *previous*, no requieren ninguna modificación, después de haberse revisado mediante "unique".

In [36]:
education = {
    'YES': 'yes',
    'Yes': 'yes',
    'NO': 'no',
    'No': 'no',
}

data['education'] = data['education'].replace(education)

In [16]:
contact = {
    'cellular': 'phone',
    'mobile': 'phone'
}

data['contact'] = data['contact'].replace(contact)

['unknown' 'telephone']


In [37]:
poutcome = {
    'UNK': 'unknown',
    'Success': 'success'
}

data['poutcome'] = data['poutcome'].replace(poutcome)

In [27]:
#df_sin_duplicados = df.drop_duplicates()